Code taken from the official BerTopic github/site: https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html \
Code for reducing outliers based on probability: https://github.com/MaartenGr/BERTopic/issues/529 \
Covert output to features: https://stackoverflow.com/questions/73768683/how-to-get-topic-probs-matrix-in-bertopic-modeling

Visual Novel BerTopic

In [ ]:
!nvidia-smi

In [ ]:
!pip install bertopic



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 75.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [ ]:
from bertopic import BERTopic
import pandas
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
import numpy as np

In [ ]:
additional = ["game", "play"]
stop = ENGLISH_STOP_WORDS.union(additional)

In [ ]:
#loading the dataset
df = pandas.read_csv("/content/novel_final_Bert.csv")

In [ ]:
df.head()

,app_id,app_name,review_text,review_score,App ID,Genre,Owners,Price,Initial Price,Required Age,Tags
0,105000,A New Beginning - Final Cut,A New Beginning - Final Cut is a point and cli...,0,105000,"Adventure, Indie","200,000 .. 500,000",999,999,0,"Adventure: 128, Point & Click: 108, Indie: 40,..."
1,105000,A New Beginning - Final Cut,"So, i read the reviews on here. Everyone is go...",0,105000,"Adventure, Indie","200,000 .. 500,000",999,999,0,"Adventure: 128, Point & Click: 108, Indie: 40,..."
2,105000,A New Beginning - Final Cut,Classic point &amp; click game with an interes...,0,105000,"Adventure, Indie","200,000 .. 500,000",999,999,0,"Adventure: 128, Point & Click: 108, Indie: 40,..."
3,105000,A New Beginning - Final Cut,"Adding together all the elements, playful and ...",0,105000,"Adventure, Indie","200,000 .. 500,000",999,999,0,"Adventure: 128, Point & Click: 108, Indie: 40,..."
4,105000,A New Beginning - Final Cut,Personal Rating: Nice Journey Classic Rating: ...,0,105000,"Adventure, Indie","200,000 .. 500,000",999,999,0,"Adventure: 128, Point & Click: 108, Indie: 40,..."


In [ ]:
review = df["review_text"].values

In [ ]:
#preperation
embedding_model = SentenceTransformer("all-mpnet-base-v2")
UMAP_model = UMAP(random_state=101)
hdbscan_model = HDBSCAN( metric='euclidean', prediction_data = True, min_samples = 3, min_cluster_size = 20)
vectorizer_model = CountVectorizer(min_df = 5, stop_words = list(stop))
ctfidf_model = ClassTfidfTransformer()

In [ ]:
#making topic model
topic_model = BERTopic(embedding_model=embedding_model, umap_model= UMAP_model, hdbscan_model= hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, calculate_probabilities=True, nr_topics=121)

In [ ]:
topics, probs = topic_model.fit_transform(review)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,4778,-1_story_like_just_really
1,0,951,0_combat_battle_jrpg_fighting
2,1,924,1_vn_visual_novel_story
3,2,809,2_bird_moon_birds_hatoful
4,3,782,3_borderlands_telltale_tales_episode
...,...,...,...
116,115,21,115_higher_magic_class_revolution
117,116,21,116_fata_morgana_house_tragedies
118,117,20,117_steve_flower_summer_shop
119,118,20,118_sierra_rendered_hours_terrible


In [ ]:
# Use the "c-TF-IDF" strategy with a threshold
probability_threshold = 0.01
new_topics = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in probs]

In [ ]:
topic_model.update_topics(review, topics=new_topics, vectorizer_model=vectorizer_model)

In [ ]:
topic_model.get_topic_info().sort_values("Count", ascending = False)

,Topic,Count,Name
2,1,3154,1_visual_vn_novel_story
5,4,1693,4_best_like_just_story
1,0,1174,0_combat_battle_enemies_jrpg
8,7,953,7_emily_away_brad_friend
3,2,803,2_bird_moon_birds_hatoful
...,...,...,...
95,94,23,94_click_matter_wedding_zombie
116,115,21,115_higher_magic_class_revolution
117,116,21,116_morgana_house_tragedies_stories
87,86,21,86_samantha_poets_romantic_wordsworth


In [ ]:
topic_model.get_topic(2)

NameError: ignored

In [ ]:
#getting output features
output_features = pandas.DataFrame(probs)
output_features["sentiment"] = df["review_score"]


In [ ]:
output_features.to_csv("features_novel_Bert_reduction.csv")